# Markov Chain

**Learning Objectives:** Learn how to build Markov Chains from n-grams and generate new sequences from the Markov Chains.

In [1]:
import types
from itertools import islice
import random
from collections import defaultdict

## n-grams

You can read about the background information related to n-grams [here](https://en.wikipedia.org/wiki/N-gram). Write a function, `build_ngrams`, that returns n-grams from an input sequene (iterator). Try to do this without concrete lists. The `islice` function may be helpful.

In [2]:
def build_ngrams(itr, n=2):
    """Return the sequence of n-grams from the source iterator."""
    for i,_ in enumerate(itr):
        if i<=len(itr)-n:
            lst = [itr[i+j] for j in range(n)]
            yield tuple(lst)

In [3]:
a = build_ngrams(range(10), n=2)
assert hasattr(a, '__iter__') and not isinstance(a, list)
al = list(a)
assert al == [(i,i+1) for i in range(9)]

b = build_ngrams(range(10), n=5)
assert hasattr(b, '__iter__') and not isinstance(b, list)
bl = list(b)
assert bl == [(i,i+1,i+2,i+3,i+4) for i in range(6)]

assert list(build_ngrams('one two three four five six seven'.split(' '), n=5)) == \
    [('one','two','three','four','five'),
     ('two','three','four','five','six'),
     ('three','four','five','six','seven')]

## Markov chain

You can read about the background of Markov Chains [here](https://en.wikipedia.org/wiki/Markov_chain). Write a function `build_chain`, that returns a Markov Chain for a sequence of n-grams. This function should return a `dict` with:

* The keys will be the source node in the Markov Chain, which is the first `n-1` elements in each n-gram, as a `tuple`.
* The values will be a *list* of possible targets in the Markov Chain. As you find new values for a single key, you will need to append to the list.

In [4]:
def build_chain(ngrams, chain=None):
    """Build a Markov chain out of an iterator of n-grams.
    
    Parameters
    ----------
    ngrams: list of n-tuples
        A list of n-grams as tuples, where the first n-1 elements are the source node
        in the Markov chain ahd the last element is the target node in the Markov chain.
    chain: dict or None
        An existing Markov chain to add ngrams to or None for a new chain.
    """
    if chain is None:
        mark = defaultdict(list)
    else:
        mark = chain
    for gram in list(ngrams):
        mark[gram[:-1]].append(gram[-1])
    return mark

In [5]:
random.seed(0)
seq1 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq1, n=3))
seq2 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq2, n=3), chain=chain)
assert chain[(0,0)]==[7, 10, 0, 3, 4]
assert chain[(4,2)]==[1, 3, 8, 3, 7, 1, 10, 2, 8]
assert len(chain.keys())==111

Write a function, `generate_sequence`, that can generate new sequences of length `m` from a trained Markov Chain (in the `dict` format described above). For the initial part of the sequence, randomly choose one of the keys in the Markov Chain `dict`.

In [6]:
import random

def generate_sequence(chain, m):
    """Generate a new sequence of length n from a Markov chain.
    
    Parameter
    ---------
    chain : dict
        A dict where the keys are the source node of the Markov chain steps and
        the values are a list of possible targets.
    m : int
        The length of the sequence to generate.
    """
    initial = random.choice(list(chain))
    lengt = len(initial)
    lst = []
    lst.extend(list(initial))
    for i in range(m-lengt):
        new = random.choice(chain[initial])
        lst.append(new)
        initial = list(initial[1:])
        initial.append(new)
        initial = tuple(initial)
    return(lst)

In [7]:
random.seed(0)
seq3 = [random.randint(0,10) for i in range(200)]
chain2 = build_chain(build_ngrams(seq1, n=3))
assert list(generate_sequence(chain2, 10))==[8, 0, 1, 8, 10, 6, 8, 4, 8, 9]
chain3 = build_chain(build_ngrams(seq1, n=5))
assert list(generate_sequence(chain3, 10))==[4, 1, 8, 5, 8, 3, 9, 8, 9, 4]

## Scrape the web to find lyrics

For this part of the exercise, you will need to find lyrics from one of your favorite bands, and use the [requests](http://docs.python-requests.org/en/latest/) and [BeautifulSoup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/) packages to scrape the lyrics from a website. Some guidance:

1. The more songs the better (many dozens).
2. No hand downloading or editing of the files, you must do everything from code.
3. Save all of the lyrics in a single text file in this directory.

I provide an example here of doing this for all of U2's lyrics. You will have to modify this code significantly for the band of your choice.

In [8]:
import requests
import re
from bs4 import BeautifulSoup
index = requests.get("http://www.lyricsfreak.com/b/beatles/")
#index = requests.get("http://www.u2.com/discography/lyrics/index/ltr/all/")
soup = BeautifulSoup(index.text, 'html.parser')
soup
#soup.find_all('div', class_='dn')[0].text
#x = re.sub(r'â\x80\x99', "'", soup.find_all('div', class_='dn')[0].text)
#re.sub(r'([A-Z])', lambda a: " " + a.group(1), x)
#s = 'start TT end FF'
#re.sub(r'([A-Z])\1', lambda pat: pat.group(1).lower(), s)
#soup.find_all('div', class_='maincont floatfix')[0].find_all('a')[3:]
#re.findall(r"beatles\\([/a-z+.0-9_]+)\\", soup.find_all('script')[1].text)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<title>The Beatles Lyrics</title>
<meta content="The Beatles Lyrics - Find all lyrics for songs such as Hey Jude, In My Life, Norwegian Wood at LyricsFreak.com" name="description">
<link href="/i/style.css?0412080619" rel="stylesheet" type="text/css">
<!--[if lte IE 6]><link rel="stylesheet" href="/i/ie6.css" type="text/css" media="screen" /><![endif]-->
<!--[if IE 7]><link rel="stylesheet" href="/i/ie7.css" type="text/css" media="screen" /><![endif]-->
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.11.0/jquery.min.js" type="text/javascript"></script>
<meta content="US" name="d_country">
<meta content="2017-12-17 18:40:56 -05:00" name="d_generated">
</meta></meta></link></meta></meta></head>
<body>
<script>

    window.UGAPP = {'o

First get the page that has an index of all the lyrics and create a list of the URLs of those pages:

In [9]:
def get_lyric_urls_1d():
    index = requests.get("http://www.lyricsfreak.com/o/one+direction/")
    soup = BeautifulSoup(index.text, 'html.parser')
    lyric_paths = [link.get('href') for link in
                   soup.find_all('div', class_='maincont floatfix')[0].find_all('a')[3:]]
    lyric_urls = ['http://www.lyricsfreak.com'+path for path in lyric_paths]
    return lyric_urls

In [10]:
def get_lyric_urls_182():
    index = requests.get("http://www.lyricsfreak.com/b/blink+182/")
    soup = BeautifulSoup(index.text, 'html.parser')
    lyric_paths = [link.get('href') for link in
                   soup.find_all('div', class_='maincont floatfix')[0].find_all('a')[3:-10]]
    lyric_urls = ['http://www.lyricsfreak.com'+path for path in lyric_paths]
    return lyric_urls

In [11]:
def get_lyric_urls_beatles():
    index = requests.get("http://www.lyricsfreak.com/b/beatles/")
    soup = BeautifulSoup(index.text, 'html.parser')
    lyric_paths = [link.get('href') for link in
                   soup.find_all('div', class_='maincont floatfix')[0].find_all('a')[3:-10]]
    lyric_paths.extend(["/b/beatles" + path for path in re.findall(r"beatles\\([/a-z+.0-9_]+)\\", soup.find_all('script')[1].text)])
    lyric_urls = ['http://www.lyricsfreak.com'+path for path in lyric_paths]
    return lyric_urls

In [12]:
lyric_urls = get_lyric_urls_1d()
lyric_urls2 = get_lyric_urls_182()
lyric_urls3 = get_lyric_urls_beatles()

Here is a function that takes the URL of a single lyric page and scrapes the actual lyric as text:

In [13]:
def get_lyric(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    x = re.sub(r'([A-Z])', lambda a: " " + a.group(1), \
           re.sub(r'â\x80\x99', "'", soup.find_all('div', class_='dn')[0].text))
    return x

In [14]:
#get_lyric('http://www.lyricsfreak.com/b/beatles/you+cant+do+that_10026179.html')

This gets all of the lyrics. Note the `time.sleep(1.0)`. When scraping websites, it is often important to throttle your requests so as to not get banned from the website.

In [15]:
import time

def get_all_lyrics(lyric_urls):
    for url in lyric_urls:
        time.sleep(1.0)
        yield get_lyric(url)

In [16]:
lyrics = get_all_lyrics(lyric_urls)
lyrics2 = get_all_lyrics(lyric_urls2)
lyrics3 = get_all_lyrics(lyric_urls3)

Now save all the lyrics to a text file:

In [17]:
with open('all_1d_lyrics.txt', 'w') as f:
    for lyric in lyrics:
        f.write(lyric.replace('\r\n', '\n'))
        f.write('\n')

In [18]:
with open('all_182_lyrics.txt', 'w') as f:
    for lyric in lyrics2:
        f.write(lyric.replace('\r\n', '\n'))
        f.write('\n')

In [20]:
#with open('all_beatles_lyrics.txt', 'w') as f:
   # for lyric in lyrics3:
       # f.write(lyric.replace('\r\n', '\n'))
#        f.write('\n')

Leave the following cell to grade your code for this section:

In [21]:
assert True

## Generate new lyrics with the Markov chain

Here is the fun part!

In [19]:
import textwrap

Here are some simple function for tokenizing the lyrics:

In [20]:
#from quicktoken import tokenize_lines, tokenize_line
PUNCTUATION = '`~!@#$%^&*()_-+={[}]|\:;"<,>.?/}\t\n'
def tokenize_line(line, punctuation=PUNCTUATION):
    """Split a string into a list of words, removing punctuation and stop words."""
    line = remove_punctuation(line.split(), punctuation)
    line = lower_words(line)
    for word in line:
        yield word
def tokenize_lines(lines, punctuation=PUNCTUATION):
    """Tokenize an iterator of lines, yielding the tokens."""
    for line in lines:
        for word in tokenize_line(line, punctuation):
            yield word
def remove_punctuation(words, punctuation=PUNCTUATION):
    """Remove punctuation from an iterator of words, yielding the results."""
    new_words = []
    for word in words:
        punc = "[" + re.escape(punctuation) + "]"
        word = re.sub(punc, " ", word)
        more_words = word.strip().split()
        for word in more_words:
            yield word
def lower_words(words):
    """Make each word in an iterator lowercase."""
    for word in words:
        yield word.lower()

Read in your lyric file, tokenize the text (no stop words!) and generate new song lyrics. Some things to think about:

* This will work best if you generate new lines of text of some finite length (10-30 words).
* Use `textwrap.wrap` to format these lines and separate them using newlines.
* To get interesting results, you may need to run it multiple times.

In [21]:
with open('all_1d_lyrics.txt', 'r') as f:
    d1 = f.read()

In [23]:
#with open('all_beatles_lyrics.txt', 'r') as f:
    #beatles = f.read()

In [24]:
with open('all_182_lyrics.txt', 'r') as f:
    b182 = f.read()
d182 = d1 + b182

In [27]:
chain = build_chain(build_ngrams(list(tokenize_line(d182)), n=2))
x = [" ".join(generate_sequence(chain, 30)) for i in range(20)]
x

["taught you every night that rule the way you like me no sleep tonite or hunger a baseball bat and frogs that you where he doesn't matter what you and",
 'okai liam am shamed lying through this all gone since you you to break it for my loneliness with me let down my own me you always be true you',
 "right' cause after the floor cause you say what is in mine so bad don't wait and i won't you and tender touches just gets it through my feet so",
 "alright bridge have is falling in the music's for you were there is how hard you say i will it's a while you're wearing that when you do it no",
 "drew a come on one weekend away what head's up all night i'm a mess today zayn you make a little closer hope that you're up up all all night",
 "okay i'm in the things i've told you you're free but why i tried to finally realize there's no superman but i live while we're only for me yeah yeah",
 "streets get outta control your eyes olivia i said the a loss you've been going crazy crazy just after par